<h1><strong>Análise de sentimentos em avaliações de e-commerce de roupas</strong></h1>


**Objetivo**:

Classificar as avaliações de e-commerce de roupas em positiva ou negativa.

*   Dataset: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

***Orientador:*** Thiago de Melo

***Orientando:*** Edson Brilhante

# Importando as bibliotecas

In [1]:
import numpy as np
import pandas as pd
import nltk
from textblob import TextBlob
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Baixando os pacotes NLTK

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

# Importando o *dataset*

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Edson Henrique/dataset/Womens Clothing E-Commerce Reviews.csv')

# Visualização do *dataset*

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Modelagem

## Seleciona as colunas '*Review Text*' e '*Rating*'

In [6]:
dataset = dataset[['Review Text', 'Rating']]
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22641 entries, 0 to 22640
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       22641 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 353.9+ KB


## Encoding

Seleciona o atributo *'Rating'* e o converte da seguinte forma:



*   1, 2 e 3: negativa
*   4 e 5: positiva

In [7]:
dataset['Rating'] = dataset['Rating'].apply(lambda x: 'pos' if x >= 4 else 'neg')
dataset['Rating'] = pd.Categorical(dataset['Rating'])

In [8]:
dataset

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,pos
1,Love this dress! it's sooo pretty. i happene...,pos
2,I had such high hopes for this dress and reall...,neg
3,"I love, love, love this jumpsuit. it's fun, fl...",pos
4,This shirt is very flattering to all due to th...,pos
...,...,...
22636,I was very happy to snag this dress at such a ...,pos
22637,"It reminds me of maternity clothes. soft, stre...",neg
22638,"This fit well, but the top was very see throug...",neg
22639,I bought this dress for a wedding i have this ...,neg


In [9]:
dataset['TextBlob'] = 0
dataset['TextBlob Rating'] = np.nan
dataset

,Review Text,Rating,TextBlob,TextBlob Rating
0,Absolutely wonderful - silky and sexy and comf...,pos,0,NaN
1,Love this dress! it's sooo pretty. i happene...,pos,0,NaN
2,I had such high hopes for this dress and reall...,neg,0,NaN
3,"I love, love, love this jumpsuit. it's fun, fl...",pos,0,NaN
4,This shirt is very flattering to all due to th...,pos,0,NaN
...,...,...,...,...
22636,I was very happy to snag this dress at such a ...,pos,0,NaN
22637,"It reminds me of maternity clothes. soft, stre...",neg,0,NaN
22638,"This fit well, but the top was very see throug...",neg,0,NaN
22639,I bought this dress for a wedding i have this ...,neg,0,NaN


In [10]:
f_categorizar_blob = lambda x: 'pos' if x > 0 else 'neg'

for i in range (0, dataset.shape[0]):
  blob = TextBlob(dataset['Review Text'].values[i])
  dataset.iloc[i,2] = blob.sentiment.polarity
  dataset.iloc[i,3] = f_categorizar_blob(blob.sentiment.polarity)
dataset

,Review Text,Rating,TextBlob,TextBlob Rating
0,Absolutely wonderful - silky and sexy and comf...,pos,0.633333,pos
1,Love this dress! it's sooo pretty. i happene...,pos,0.339583,pos
2,I had such high hopes for this dress and reall...,neg,0.073675,pos
3,"I love, love, love this jumpsuit. it's fun, fl...",pos,0.550000,pos
4,This shirt is very flattering to all due to th...,pos,0.512891,pos
...,...,...,...,...
22636,I was very happy to snag this dress at such a ...,pos,0.552667,pos
22637,"It reminds me of maternity clothes. soft, stre...",neg,0.091667,pos
22638,"This fit well, but the top was very see throug...",neg,0.414286,pos
22639,I bought this dress for a wedding i have this ...,neg,0.322222,pos


In [11]:
dataset['TextBlob Rating'] = pd.Categorical(dataset['TextBlob Rating'])

In [12]:
y_true = dataset['Rating'].values
y_pred = dataset['TextBlob Rating'].values

In [13]:
print('Matriz de confusão')
print(confusion_matrix(y_true, y_pred))

print('\nMétricas')
print(classification_report(y_true, y_pred, digits=3))

print('\nScore')
print(accuracy_score(y_true, y_pred))

Matriz de confusão
[[  914  4279]
 [  500 16948]]

Métricas
              precision    recall  f1-score   support

         neg      0.646     0.176     0.277      5193
         pos      0.798     0.971     0.876     17448

    accuracy                          0.789     22641
   macro avg      0.722     0.574     0.577     22641
weighted avg      0.764     0.789     0.739     22641


Score
0.7889227507618921


In [14]:
dataset[dataset['TextBlob Rating'] == 'pos']['TextBlob Rating'].count()

21227

In [15]:
dataset[dataset['TextBlob Rating'] == 'neg']['TextBlob Rating'].count()

1414